## **Descrição do projeto**
> O projeto tem o foco em desenvolver um modelo capaz de medir a taxa de evasão dos alunos do curso de Ciência da Computação da Universidade Federal de Viçosa - Campus Florestal,  onde são classificados de acordo com alguns parâmetros como idade, gênero, data de ingresso no curso, raça, cidade onde vivia antes de se mudar para as aulas, nota obtida no ENEM, CRA, modalidade de ingresso e o número de reprovações.

## **Bibliotecas utilizadas**

In [1]:
!pip install python-Levenshtein
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### NÃO REMOVA ESSA CÉLULA!! ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## **Leitura dos dados**

In [ ]:
### NÃO REMOVA ESSA CÉLULA!! ###
!wget https://raw.githubusercontent.com/ThulioSilva13/ProjetoCienciaDeDados/main/dadosEstudantesUFV.csv

--2023-05-18 19:26:00--  https://raw.githubusercontent.com/ThulioSilva13/ProjetoCienciaDeDados/main/dadosEstudantesUFV.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-05-18 19:26:01 ERROR 404: Not Found.



In [ ]:
dados = pd.read_csv('./dadosEstudantesUFV.csv')
dados

FileNotFoundError: ignored

## **Análise inicial dos dados**
    

> Inicialmente foi feita uma análise inicial dos atributos, observando fatores como o nome da coluna, o tipo de dado, os valores, dentre outros aspectos. Abaixo são listadas as observações:

> 1. Ao analisar os dados, percebeu-se que alguns possuem incompatibilidade em alguns campos, como por exemplo:
 - Não são todos os alunos que possuem valores na coluna contendo a nota do ENEM
 - Alguns campos que deveriam ter o preenchimento obrigatório, como as colunas Sexo e UF_Nascimento, estão valores NaN.
  - Alguns alunos possuem modalidade de ingresso com valor igual a zero. Desse modo, após pesquisar sobre o significado da modadlidade 0, descobriu-se que se refere ao campo de "Ampla Concorrência", entretanto, o valor de modalidade 9 também se refere a esse mesmo campo. Portanto, vale observar essa semelhança entre os dados para que futuramente no projeto evite falhas na observação e conclusão na análise desses dados.

> 2. Outra observação realizada, foi a relação entre a modalidade de ingresso e a nota obtida no ENEM, alguns alunos tem o campo preenchido com a nota do ENEM que usaram quando entraram na universidade, mas porém não consta em qual modalidade ele foi classificado. O mesmo se aplica no caso contrário, alguns alunos não possuem a nota do ENEM, mas na coluna da modalidade de ingresso, consta um valor.

> 3. Ao examinarmos o campo Situacao_Aluno, é possível perceber que ele possui diversos tipos de valores, como por exemplo, Conclusão, Desligamento, Abandono, Transferência, entre outros. Já ao observarmos o campo Situacao_Aluno_Agrupado, os valores se resumem a Conclusão, Evasão, Retenção e Matriculado. Após análises, percebeu-se que uma coluna tem relação direta com a outra, pois:
  - O valor Conclusão no campo Situacao_Aluno_Agrupado, se refere também aos valores Conclusão de Situacao_Aluno que indicam que o aluno concluiu o curso
  - Evasão corresponde aos alunos que abandonaram o curso
  - Matriculado caracteriza os alunos que ainda estão cursando
  - Retenção se refere aos alunos que ainda estão cursando, mas que por algum motivo, não estão tendo aulas

>   Foi concluído então que o campo Situacao_Aluno_Agrupado é uma versão resumida do campo Situacao_Aluno, com valores agrupados que indicam a situação atual do aluno no curso. Essa análise é importante para compreender melhor os dados e para tomar decisões com base nessas informações, além de que entender a relação entre as diferentes colunas pode ajudar a identificar tendências.

## **Perguntas que guiarão o projeto**

> Inicialmente, existem 10 perguntas que guiarão o projeto e que pretendemos responder durante o desenvolvimento do modelo, sendo elas:

 -  O CRA segue uma distribuição normal?
 (Histograma)

 - Avaliar se o gênero influencia na evasão através de análises da porcentagem de meninas que evadiram em comparação com o valor percentual de meninos que evadiram.
 (Teste de hipótese com algumas estatísticas descritivas)

 - Número médio de alunos que ingressaram no curso durante a pandemia de COVID-19, onde se espera entender se o método de ensino remoto influenciou ou não na decisão de saída do curso.
 (Gráfico de linhas)

 - Esperamos prever a quantidade de alunos, em porcentagem, por modalidade de ingresso abandonaram o curso.
 (Gráfico de pizza)

 - Visamos compreender se alunos que vieram de escolas públicas encontram mais dificuldades e isso os levam a abandonar o curso.
 (Teste de hipótese)

 - Visamos calcular o valor médio de reprovações que os alunos que evadiram tiveram e analisar se há um padrão ou intervalo de números que aparecem de forma frequente.
 (Estatística descritiva, calcular média)

 - Qual a média de idade das pessoas que decidiram abandonar o curso. O aluno ser mais velho ou mais novo influencia na decisão de saída?
 (Estatística descritiva, calcular média)

 - Existe alguma relação entre a nota do ENEM e a evasão do curso?
(Comparar Gráficos entre relação nota e evasão)


 - É possível afirmar que os alunos abandonam o curso por serem de algum estado diferente de Minas Gerais, onde o campus se localiza?
(Gráfico e estatística descritiva)


 - Realizar o cálculo de estatísticas descritivas sobre o CRA de alunos que abandonaram o curso em comparação com o CRA de alunos que concluíram e dessa forma, tentamos estabelecer uma relação de influência entre o CRA e a evasão.
 

 - Através do cálculo da média do CRA dos alunos de Ciência da Computação, esperamos ser possível classificar se uma amostra pertence ou não a população que representa os alunos do curso citado.

1) Criar um dataFrame para salvar apenas alunos que já saíram do curso.
2) Verificar campos que estejam com valores NULL ou NaN
3) Definir uma valor só pra ampla concorrência(ao invés de 0 e 9)
4) Cursos que são o mesmo porém com dois ou mais nomes diferentes
5) Remover alunos que não ingressaram via ENEM
6) Remover alunos que ingressaram sem nenhuma modalidade
7) Remover cidades com problemas 

## **Formatação e limpeza dos dados**

Essa etapa envolve entender os atributos e objetos dos dados, os tipos de cada atributo, o domínio de cada atributo,
verificar e identificar possíveis ruídos ou informações ausentes, criar novos atributos se necessário, formatar valores, juntar conjuntos de dados, dentre outras atividades.

In [ ]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def standardize_strings(column, threshold=80):
    standardized_strings = []
    for string in column:
        standardized = False
        for standard in standardized_strings:
            similarity = fuzz.ratio(string, standard)
            if similarity >= threshold:
                column.replace(string, standard, inplace=True)
                standardized = True
                break
        
        if not standardized:
            standardized_strings.append(string)

    return column

def merge_entries(course_name):
    if course_name.endswith(' - Licenciatura'):
        return course_name[:-len(' - Licenciatura')]
    else:
        return course_name

def remove_suffix(entry):
    return entry.split(' - ')[0]

df = pd.read_csv('a.csv',header=0)
print(df.columns.values)

# remove valores nulos
df = df.dropna()

# na database, existem alunos nascidos após 2015
# supos-se que isso não faz sentido
df = df[df["Ano_Nascimento"] < 2015]

# alunos cujo campo de estado estão em branco
# foram removidos para propositos da análise
df= df[df["UF_Nascimento"] != " "]

# remove acentos, espaços e passa tudo para lowercase
df['Municipio_Nascimento'] = df['Municipio_Nascimento'].apply(lambda x: strip_accents(x).strip().lower())
df['Situacao_Aluno'] = df['Situacao_Aluno'].apply(lambda x: strip_accents(x).strip().lower())

# une cursos de licenciatura com sufixo específico ' - Licenciatura'
# proprio de cursos em Florestal
df['Curso'] = df['Curso'].apply(merge_entries)
df['Curso'] = df['Curso'].apply(lambda x: x.strip())

# renomeia municipios com sufixos de estado
# ex "vicosa", "vicosa - mg" 
df['Municipio_Nascimento'] = df['Municipio_Nascimento'].apply(remove_suffix)

# cria um df onde só existem alunos que evadiram ou formam
df_filtro = df[(df["Situacao_Aluno_Agrupada"] == "Conclusão") | (df["Situacao_Aluno_Agrupada"] == "Evasão")]

# une municipios de nome similar a partir de threshold
# ex: "vcosa", "vicosa"
standardized_column = standardize_strings(df['Municipio_Nascimento'], threshold=80)
df['Municipio_Nascimento'] = standardized_column

'''
res = []
for i in (df["Ano_Nascimento"].unique()):
	res.append(i)

print(sorted(res))
'''

## **Estatísticas descritivas sobre os dados**



Nesta etapa, o grupo também irá gerar estatísticas descritivas, gráficos e tabelas para conhecer os dados. Todo conhecimento importante extraído deverá ser documentado e discutido. Nesta etapa, o objetivo é responder parte das perguntas elaboradas. Lembrem-se que novos questionamentos podem surgir e devem ser documentados.